In [67]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [68]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [69]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [70]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7719


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""03/04/2025""","""4175""","""11 06 05 20 12""","""0506111220""","[5, 6, … 20]",5,6,11,12,20,"""05""","""06""","""11""","""12""","""20""",null,null,true,16,"""01_05_01_08""",19
"""02/04/2025""","""4174""","""26 14 24 11 17""","""1114172426""","[11, 14, … 26]",11,14,17,24,26,"""11""","""14""","""17""","""24""","""26""","""05""","""20""",true,24,"""03_03_07_02""",-13
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""26""",false,0,"""04_01_12_08""",-29
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""33""",false,0,"""09_06_04_08""",5
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8
"""29/03/2025""","""4170""","""23 06 04 09 10""","""0406091023""","[4, 6, … 23]",4,6,9,10,23,"""04""","""06""","""09""","""10""","""23""","""03""","""35""",false,0,"""02_03_01_13""",17
"""28/03/2025""","""4169""","""20 10 01 09 29""","""0109102029""","[1, 9, … 29]",1,9,10,20,29,"""01""","""09""","""10""","""20""","""29""","""04""","""23""",true,16,"""08_01_10_09""",-26
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42
"""26/03/2025""","""4167""","""34 29 06 24 26""","""0624262934""","[6, 24, … 34]",6,24,26,29,34,"""06""","""24""","""26""","""29""","""34""","""05""","""35""",true,16,"""18_02_03_05""",-16


In [ ]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  # chart_unijump = df_jumps_map.hvplot.bar(
  #   x='jumps_map',
  #   y='count'
  # )

  display(df_viz)
  display(chart)
  display(df_10_last)
  display(df_jumps_map)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list()
  return lst_post_str_b, lst_jumps_map

In [72]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1', test_value="05")

post_str_b1,count,percent
str,u32,f64
"""02""",90,13.353116
"""01""",89,13.204748
"""03""",70,10.385757
"""05""",68,10.089021
"""04""",63,9.347181
…,…,…
"""19""",1,0.148368
"""28""",1,0.148368
"""22""",1,0.148368


:Overlay
   .Bars.I  :Bars   [post_str_b1]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42
"""09/02/2025""","""4122""","""05 21 20 35 29""","""0520212935""","[5, 20, … 35]",5,20,21,29,35,"""05""","""20""","""21""","""29""","""35""","""07""","""32""",false,0,"""15_01_08_06""",-30
"""10/01/2025""","""4092""","""06 14 05 13 20""","""0506131420""","[5, 6, … 20]",5,6,13,14,20,"""05""","""06""","""13""","""14""","""20""","""02""","""27""",false,0,"""01_07_01_06""",23
"""09/01/2025""","""4091""","""14 32 17 05 06""","""0506141732""","[5, 6, … 32]",5,6,14,17,32,"""05""","""06""","""14""","""17""","""32""","""05""","""20""",false,0,"""01_08_03_15""",29
"""25/12/2024""","""4076""","""14 21 18 28 05""","""0514182128""","[5, 14, … 28]",5,14,18,21,28,"""05""","""14""","""18""","""21""","""28""","""10""","""32""",true,16,"""09_04_03_07""",1
"""22/12/2024""","""4073""","""19 20 35 07 05""","""0507192035""","[5, 7, … 35]",5,7,19,20,35,"""05""","""07""","""19""","""20""","""35""","""06""","""32""",false,0,"""02_12_01_15""",46
"""21/12/2024""","""4072""","""17 34 05 28 29""","""0517282934""","[5, 17, … 34]",5,17,28,29,34,"""05""","""17""","""28""","""29""","""34""","""05""","""35""",false,0,"""12_11_01_05""",23
"""19/12/2024""","""4070""","""34 05 22 18 30""","""0518223034""","[5, 18, … 34]",5,18,22,30,34,"""05""","""18""","""22""","""30""","""34""","""07""","""27""",false,0,"""13_04_08_04""",-21
"""16/12/2024""","""4067""","""14 29 05 11 07""","""0507111429""","[5, 7, … 29]",5,7,11,14,29,"""05""","""07""","""11""","""14""","""29""","""02""","""26""",true,17,"""02_04_03_15""",16


jumps_map,count
str,u32
"""05_01_09_03""",3
"""05_08_04_13""",2
"""02_04_11_13""",2
"""04_03_01_05""",2
"""03_05_07_12""",2
…,…
"""07_13_01_04""",1
"""06_02_10_04""",1
"""05_02_10_02""",1


In [73]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast', test_value="20")

post_str_blast,count,percent
str,u32,f64
"""33""",13,14.942529
"""34""",12,13.793103
"""35""",11,12.643678
"""31""",7,8.045977
"""30""",6,6.896552
…,…,…
"""17""",2,2.298851
"""22""",1,1.149425
"""18""",1,1.149425


:Overlay
   .Bars.I  :Bars   [post_str_blast]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""10/01/2025""","""4092""","""06 14 05 13 20""","""0506131420""","[5, 6, … 20]",5,6,13,14,20,"""05""","""06""","""13""","""14""","""20""","""02""","""27""",false,0,"""01_07_01_06""",23
"""15/09/2024""","""3975""","""18 08 17 09 20""","""0809171820""","[8, 9, … 20]",8,9,17,18,20,"""08""","""09""","""17""","""18""","""20""","""03""","""30""",true,16,"""01_08_01_02""",22
"""17/01/2024""","""3733""","""11 15 04 09 20""","""0409111520""","[4, 9, … 20]",4,9,11,15,20,"""04""","""09""","""11""","""15""","""20""","""01""","""31""",true,16,"""05_02_04_05""",-6
"""28/08/2023""","""3591""","""20 08 11 04 14""","""0408111420""","[4, 8, … 20]",4,8,11,14,20,"""04""","""08""","""11""","""14""","""20""","""21""","""33""",true,24,"""04_03_03_06""",2
"""10/07/2023""","""3542""","""02 20 07 03 15""","""0203071520""","[2, 3, … 20]",2,3,7,15,20,"""02""","""03""","""07""","""15""","""20""","""02""","""17""",false,0,"""01_04_08_05""",-8
"""13/09/2022""","""3242""","""11 18 08 20 05""","""0508111820""","[5, 8, … 20]",5,8,11,18,20,"""05""","""08""","""11""","""18""","""20""","""02""","""30""",true,24,"""03_03_07_02""",-13
"""28/08/2022""","""3226""","""04 12 18 07 20""","""0407121820""","[4, 7, … 20]",4,7,12,18,20,"""04""","""07""","""12""","""18""","""20""","""02""","""26""",true,16,"""03_05_06_02""",-4
"""07/08/2022""","""3205""","""02 20 03 17 11""","""0203111720""","[2, 3, … 20]",2,3,11,17,20,"""02""","""03""","""11""","""17""","""20""","""06""","""34""",true,1,"""01_08_06_03""",8
"""08/03/2022""","""3053""","""06 10 02 05 20""","""0205061020""","[2, 5, … 20]",2,5,6,10,20,"""02""","""05""","""06""","""10""","""20""","""12""","""30""",true,16,"""03_01_04_10""",-2


jumps_map,count
str,u32
"""12_03_01_03""",2
"""01_01_01_07""",1
"""07_07_01_01""",1
"""05_03_02_02""",1
"""02_05_02_07""",1
…,…
"""05_02_02_05""",1
"""03_03_07_02""",1
"""01_03_06_05""",1


In [74]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [75]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))

TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [76]:
df_Orows.filter(is_rare_combination=False)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,"""01""","""02""","""04""","""05""","""13""","""01""","""12""",false,0,"""01_02_01_08""",10
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2829313435""","[28, 29, … 35]",28,29,31,34,35,"""28""","""29""","""31""","""34""","""35""","""28""","""35""",false,0,"""01_02_03_01""",-3
"""2830313334""","[28, 30, … 34]",28,30,31,33,34,"""28""","""30""","""31""","""33""","""34""","""28""","""35""",false,0,"""02_01_02_01""",-4
"""2830313435""","[28, 30, … 35]",28,30,31,34,35,"""28""","""30""","""31""","""34""","""35""","""28""","""35""",false,0,"""02_01_03_01""",-7


In [ ]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  pl.col("is_rare_combination") == False
)

df_possible

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
"""0102040533""","[1, 2, … 33]",1,2,4,5,33,"""01""","""02""","""04""","""05""","""33""","""01""","""32""",false,0,"""01_02_01_28""",30
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1029313435""","[10, 29, … 35]",10,29,31,34,35,"""10""","""29""","""31""","""34""","""35""","""10""","""35""",false,0,"""19_02_03_01""",-21
"""1029323335""","[10, 29, … 35]",10,29,32,33,35,"""10""","""29""","""32""","""33""","""35""","""10""","""34""",false,0,"""19_03_01_02""",-11
"""1029323435""","[10, 29, … 35]",10,29,32,34,35,"""10""","""29""","""32""","""34""","""35""","""10""","""35""",false,0,"""19_03_02_01""",-15


In [108]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0306101629""","[3, 6, … 29]",3,6,10,16,29,"""03""","""06""","""10""","""16""","""29""","""03""","""28""",false,0,"""03_04_06_13""",4
